In [1]:
!pip install openai>=1.0.0 datasets>=2.0.0


In [2]:
from datasets import load_dataset

dataset = load_dataset("allenai/ai2_arc", "ARC-Challenge")
val_data = dataset["validation"]

len(val_data)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

ARC-Challenge/train-00000-of-00001.parqu(…):   0%|          | 0.00/190k [00:00<?, ?B/s]

ARC-Challenge/test-00000-of-00001.parque(…):   0%|          | 0.00/204k [00:00<?, ?B/s]

ARC-Challenge/validation-00000-of-00001.(…):   0%|          | 0.00/55.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1172 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/299 [00:00<?, ? examples/s]

299

In [3]:
import string

def format_prompt(example):
    question = example["question"]
    choices = example["choices"]["text"]
    labels = example["choices"]["label"]

    lines = [question]
    for label, text in zip(labels, choices):
        lines.append(f"{label}. {text}")

    return "\n".join(lines)


In [ ]:
import os

openai.api_key = os.getenv("OPENAI_API_KEY")


In [8]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [9]:
print(format_prompt(val_data[0]))


Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?
A. Put the objects in groups.
B. Change the height of the ramp.
C. Choose different objects to roll.
D. Record the details of the investigation.


In [11]:
response = client.responses.create(
    model="ft:gpt-4.1-nano-2025-04-14:algoverse-ai-safety:arc-v2:CrBuBGfj",
    input=format_prompt(val_data[0]),
    temperature=0,
    include=["message.output_text.logprobs"]
)


In [12]:
response.output


[ResponseOutputMessage(id='msg_07013ec9d484b20800695b586e60688193aaf4c4408e1a275c', content=[ResponseOutputText(annotations=[], text='D', type='output_text', logprobs=[Logprob(token='D', bytes=[68], logprob=-4e-05, top_logprobs=[])])], role='assistant', status='completed', type='message')]

In [13]:
pred = response.output[0].content[0].text.strip()


In [14]:
import math

first_logprob = response.output[0].content[0].logprobs[0].logprob
confidence = math.exp(first_logprob)


In [15]:
print(pred)
print(confidence)


D
0.9999600007999894


In [17]:
import math
from tqdm import tqdm
# Storage
correct = 0
total = len(val_data)
brier_scores = []
confidences = []
correctnesses = []

# Loop over all 299 ARC validation questions
for example in tqdm(val_data):
    # Ground truth
    gold = example["answerKey"].strip()
    # Model call
    response = client.responses.create(
        model="ft:gpt-4.1-nano-2025-04-14:algoverse-ai-safety:arc-v2:CrBuBGfj",
        input=format_prompt(example),
        temperature=0,
        include=["message.output_text.logprobs"]
    )
    # Extract prediction
    pred = response.output[0].content[0].text.strip()
    # Extract confidence from first token logprob
    first_logprob = response.output[0].content[0].logprobs[0].logprob
    confidence = math.exp(first_logprob)
    # Correctness
    is_correct = 1 if pred == gold else 0
    # Accuracy tracking
    correct += is_correct
    # Brier score
    brier_scores.append((confidence - is_correct) ** 2)
    # Store for selective metrics
    confidences.append(confidence)
    correctnesses.append(is_correct)

# ---- Final metrics ----
accuracy = correct / total
brier_score = sum(brier_scores) / total
# Selective accuracy @ 99%
threshold = 0.99
selected = [ correctnesses[i] for i in range(total) if confidences[i] >= threshold ]
coverage = len(selected) / total
selective_accuracy = sum(selected) / len(selected) if len(selected) > 0 else float("nan")
# Print final answers
print("Accuracy:", accuracy)
print("Brier Score:", brier_score)
print("Selective Accuracy @ 99%:", selective_accuracy)
print("Coverage @ 99%:", coverage)

100%|██████████| 299/299 [02:48<00:00,  1.77it/s]

Accuracy: 0.903010033444816
Brier Score: 0.09772771021915366
Selective Accuracy @ 99%: 0.927007299270073
Coverage @ 99%: 0.9163879598662207
